In [1]:
import os
import sys
#path environment
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")
import traci

import xml.etree.ElementTree as ET

In [2]:
sumo_cmd = ['sumo', 
            '--duration-log.statistics', 
            '--tripinfo-output', 'my_output_file.xml', 
            '-c', 'maps/grid/grid.sumo.cfg']

In [3]:
def analyze_time_loss(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Initialize variables for analysis
    total_time_loss = 0.0
    total_waiting_time = 0.0
    total_duration = 0.0
    total_reroutes = 0
    vehicle_count = 0

    # Iterate over each tripinfo element in the XML
    for trip in root.findall('tripinfo'):
        time_loss = float(trip.get('timeLoss'))
        waiting_time = float(trip.get('waitingTime'))
        duration = float(trip.get('duration'))
        reroutes = int(trip.get('rerouteNo'))
        
        total_time_loss += time_loss
        total_waiting_time += waiting_time
        total_duration += duration
        total_reroutes += reroutes
        vehicle_count += 1

    # Calculate averages
    average_time_loss = total_time_loss / vehicle_count if vehicle_count > 0 else 0
    average_waiting_time = total_waiting_time / vehicle_count if vehicle_count > 0 else 0
    average_duration = total_duration / vehicle_count if vehicle_count > 0 else 0
    average_reroutes = total_reroutes / vehicle_count if vehicle_count > 0 else 0

    # Store results in a dictionary
    results = {
        'Total Vehicles': vehicle_count,
        'Total Time Loss': total_time_loss,
        'Average Time Loss': average_time_loss,
        'Total Waiting Time': total_waiting_time,
        'Average Waiting Time': average_waiting_time,
        'Average Duration': average_duration,
        'Total Reroutes': total_reroutes,
        'Average Reroutes': average_reroutes
    }
    return results


In [4]:
# Start SUMO with TraCI support
traci.start(sumo_cmd)
step = 0
while step < 3600:
    traci.simulationStep()
    
    
    step += 1

traci.close()


 Retrying in 1 seconds
***Starting server on port 46049 ***
Loading net-file from 'maps/grid/grid.net.xml' ... done (75ms).
Loading done.
Simulation version 1.18.0 started with time: 0.00.
Step #3600.00 (1ms ~= 1000.00*RT, ~148000.00UPS, TraCI: 1ms, vehicles TOT 3600 ACT 148 BUF
Simulation ended at time: 3600.00
Reason: TraCI requested termination.
Performance: 
 Duration: 6.47s
 TraCI-Duration: 0.86s
 Real time factor: 556.758
 UPS: 78606.866687
Vehicles: 
 Inserted: 3600
 Running: 148
 Waiting: 0
Statistics (avg of 3452):
 RouteLength: 1629.94
 Speed: 11.36
 Duration: 143.71
 WaitingTime: 2.53
 TimeLoss: 23.13
 DepartDelay: 0.35

DijkstraRouter answered 3600 queries and explored 182.34 edges on average.
DijkstraRouter spent 0.85s answering queries (0.24ms on average).


In [5]:
results = analyze_time_loss('my_output_file.xml')


In [6]:
results

{'Total Vehicles': 3452,
 'Total Time Loss': 79836.51000000018,
 'Average Time Loss': 23.12761008111245,
 'Total Waiting Time': 8731.0,
 'Average Waiting Time': 2.5292584009269987,
 'Average Duration': 143.7111819235226,
 'Total Reroutes': 3426,
 'Average Reroutes': 0.992468134414832}